In [1]:
# | default_exp classes.DomoUser

In [2]:
#| export
from fastcore.basics import patch, patch_to

In [3]:
# | exporti
from dataclasses import dataclass, field
from typing import List, Optional

import domolibrary.utils.DictDot as util_dd
import domolibrary.client.DomoAuth as dmda
import domolibrary.client.Logger as lc
import domolibrary.routes.user as user_routes

In [4]:
# | export
@dataclass
class DomoUser:
    id: str
    display_name: str = None
    email_address: str = None
    role_id: str = None

    publisher_domain: str = None
    subscriber_domain: str = None
    virtual_user_id: str = None

    auth: Optional[dmda.DomoAuth] = field(repr=False, default=None)

    @classmethod
    def _from_search_json(cls, auth, user_obj):
        user_dd = util_dd.DictDot(user_obj)

        return cls(
            auth=auth,
            id=str(user_dd.id) or str(user_dd.userId),
            display_name=user_dd.displayName,
            email_address=user_dd.emailAddress,
            role_id=user_dd.roleId
        )

    @classmethod
    def _from_virtual_json(cls, auth, user_obj):
        user_dd = util_dd.DictDot(user_obj)

        return cls(
            id=user_dd.id,
            auth=auth,
            publisher_domain=user_dd.publisherDomain,
            subscriber_domain=user_dd.subscriberDomain,
            virtual_user_id=user_dd.virtualUserId
        )

In [5]:
@patch_to(DomoUser)
async def reset_password(self: DomoUser, new_password: str,
                         debug_api: bool = False):

    res = await user_routes.reset_password(auth=self.auth,
                                           user_id=self.id,
                                           new_password=new_password,
                                           debug_api=debug_api)

    return res


In [6]:
@patch_to(DomoUser, cls_method=True)
async def request_password_reset(cls, domo_instance: str,
                                 email: str,
                                 locale: str = 'en-us',
                                 debug_api: bool = False):

    return await user_routes.request_password_reset(domo_instance=domo_instance, email=email, locale=locale,
                                                    debug_api=debug_api)


In [7]:
@patch_to(DomoUser, cls_method=True)
async def create_user(cls : DomoUser, full_auth: dmda.DomoFullAuth, display_name, email, role_id, password: str = None,
                        send_password_reset_email: bool = False,
                        debug: bool = False, log_results: bool = False):

    res = await user_routes.create_user(full_auth=full_auth, display_name=display_name, email=email,
                                        role_id=role_id, debug=debug, log_results=log_results)

    if debug:
        print(res)

    if res.status != 200:
        return None

    dd = util_dd.DictDot(res.response)
    u = cls(domo_instance=full_auth.domo_instance,
            full_auth=full_auth,
            id=dd.id or dd.userId,
            display_name=dd.displayName,
            email_address=dd.emailAddress)

    if password:
        await u.reset_password(new_password=password)

    if send_password_reset_email:
        await u.request_password_reset(domo_instance=full_auth.domo_instance,
                                        email=u.email_address)

    return u
    
async def set_user_landing_page(self, 
                                page_id:str,
                                user_id: str = None,
                                full_auth : dmda.DomoFullAuth = None,
                                debug:bool = False):

    res = await user_routes.set_user_landing_page(full_auth = full_auth or self.full_auth, page_id = page_id, 
                                                    user_id = self.id or user_id , debug = debug)

    if res.status != 200:
        return False

    return True

In [8]:
# | export

@dataclass
class DomoUsers:
    logger : Optional[lc.Logger] = None

    def _users_to_domo_user(user_ls, auth: dmda.DomoAuth):
        domo_users = []
        for user_obj in user_ls:
            user = DomoUser._from_search_json(auth=auth, user_obj=user_obj)
            domo_users.append(user)

        return domo_users

    def _users_to_virtual_user(user_ls, auth: dmda.DomoAuth):
        domo_users = []
        for user_obj in user_ls:
            user = DomoUser._from_virtual_json(auth=auth, user_obj=user_obj)
            domo_users.append(user)

        return domo_users

    def _generate_logger(self, logger : Optional[lc.Logger] = None):
        self.logger = logger or self.logger or lc.loger()


In [9]:
@patch_to(DomoUsers, cls_method=True)
async def all_users(cls: DomoUsers, auth: dmda.DomoAuth,
                    debug_api: bool = False,
                    debug_prn: bool = False,
                    debug_log: bool = False,
                    logger: Optional[lc.Logger] = None
                    ) -> [DomoUser]:
    
    logger = logger or lc.Logger( app_name = 'all_users')

    res = await user_routes.get_all_users(
        auth=auth, debug_api = debug_api
    )

    if not res.is_success:
        return None

    users_ls = res.response

    message = f"{len(users_ls)} users retrieved from {auth.domo_instance}"
    
    if debug_prn:
        print(message)    
    logger.log_info(message = message, debug_log = debug_log)
        
    return cls._users_to_domo_user(user_ls=users_ls, auth=auth)


In [10]:
import os
import pandas as pd

token_auth = dmda.DomoTokenAuth(domo_instance='domo-dojo',
                                domo_access_token=os.environ['DOMO_DOJO_ACCESS_TOKEN'])

domo_users = await DomoUsers.all_users(auth=token_auth, debug_api = False, debug_prn=False)

pd.DataFrame(domo_users[0:3])


,id,display_name,email_address,role_id,publisher_domain,subscriber_domain,virtual_user_id,auth
0,0,monitor,monitor@domo.com,1,None,None,None,"{'domo_instance': 'domo-dojo', 'domo_access_to..."
1,1006847540,Marc-Anton Clavel,marcanton.clavel@domo.com,2,None,None,None,"{'domo_instance': 'domo-dojo', 'domo_access_to..."
2,1012895591,JeMiller,JeMiller@marketaxess.com,2097317660,None,None,None,"{'domo_instance': 'domo-dojo', 'domo_access_to..."


In [11]:
@patch_to(DomoUsers, cls_method=True)
async def by_id(
    cls: DomoUsers,
    user_ids: list[str],
    auth: dmda.DomoAuth,
    only_allow_one: bool = True,
    debug_api: bool = False,
    return_raw: bool = False
) -> list:

    body = user_routes.generate_search_users_body_by_id(user_ids)

    res = await user_routes.search_users(
        process_users=False,
        body=body,
        debug_api=debug_api, auth=auth,
    )

    if return_raw:
        return res

    if not res.is_success:
        return None

    user_ls = res.response.get("users")

    domo_users = cls._users_to_domo_user(user_ls = user_ls,
                                         auth=auth)

    if only_allow_one:
        return domo_users[0]

    return domo_users


In [12]:
import os
import pandas as pd

token_auth = dmda.DomoTokenAuth(domo_instance='domo-dojo',
                                domo_access_token=os.environ['DOMO_DOJO_ACCESS_TOKEN'])


await DomoUsers.by_id(auth=token_auth,
                      user_ids=['1006847540', '1012895591'],
                      only_allow_one = True,
                      return_raw=False
                      )


DomoUser(id='1012895591', display_name='JeMiller', email_address='JeMiller@marketaxess.com', role_id=2097317660, publisher_domain=None, subscriber_domain=None, virtual_user_id=None)

### Search by Email

In [13]:
@patch_to(DomoUsers, cls_method = True)
def util_match_domo_users_to_emails(cls : DomoUsers, domo_users: list,
user_email_ls: List[str]) -> list:
    """pass in an array of user emails to match against an array of Domo User"""

    matches = []
    for idx, email in enumerate(user_email_ls):
        match_user = next((
                domo_user
                for domo_user in domo_users
                if email.lower() == domo_user.email_address.lower()
            ), None)
        if match_user:
            matches.append(match_user)
    return matches


In [14]:
@patch_to(DomoUsers, cls_method = True)
async def by_email(
    cls : DomoUsers,
    user_email_ls: list,
    auth: dmda.DomoAuth,
    only_allow_one: bool = True,
    debug_api: bool = False,
    return_raw : bool = False
) -> list:

    body = user_routes.generate_search_users_body_by_email(
        user_email_ls=user_email_ls)
    res = await user_routes.search_users(
        full_auth=full_auth, body=body, log_results=log_results, debug=debug
    )

    if res.status == 200:
        json_list = res.response.get("users")

        if not json_list:
            return None

        domo_users = cls._users_to_domo_user(user_ls, full_auth=full_auth)

        if only_allow_one:
            domo_users = cls.util_match_domo_users_to_emails(
                domo_users, [email_address]
            )
            return domo_users[0]

        return domo_users

    return None

In [15]:
@classmethod
async def virtual_user_by_subscriber_instance(
    cls,
    subscriber_instance: str,
    auth: dmda.DomoAuth,
    debug_api: bool = False,
    return_raw : bool = False
):
    res = await user_routes.search_virtual_user_by_subscriber_instance(
        auth=auth,
        subscriber_instance=subscriber_instance,
        debug_api=debug_api,
    )

    if return_raw:
        return res

    if res.status != 200:
        return None

    json_list = res.response

    if debug:
        print(json_list)

    if not json_list:
        return None

    domo_users = cls._users_to_virtual_user(json_list, full_auth=full_auth)
    return domo_users[0]